In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### Extract Text from PDF Papers

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data/PDFs').load_data()
index = VectorStoreIndex.from_documents(documents)

### Generate Questions

In [ ]:
from llama_index.evaluation import DatasetGenerator

data_generator = DatasetGenerator.from_documents(documents)
questions = data_generator.generate_questions_from_nodes()
questions

In [ ]:
with open('data/llama_index_generated_questions.txt', 'w') as wf:
    for q in questions:
        wf.write("%s\n" % q)

### Evaluation

##### Are questions, answers, and context matching?

In [ ]:
import random

sampled_questions = random.sample(questions, 100)

In [ ]:
import pandas as pd
import time
from llama_index.evaluation import ResponseEvaluator
from llama_index.evaluation import QueryResponseEvaluator
from llama_index.llm_predictor import LLMPredictor
from langchain.chat_models import ChatOpenAI
from llama_index import ServiceContext

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
r_evaluator = ResponseEvaluator(service_context=service_context)
qr_evaluator = QueryResponseEvaluator(service_context=service_context)

query_engine = index.as_query_engine()
questions_and_answers = []
for q in sampled_questions:
    a = query_engine.query(q)
    r_eval_result = r_evaluator.evaluate(a)
    time.sleep(8)
    qr_eval_result = qr_evaluator.evaluate(q, a)
    questions_and_answers.append({'Question': q, 'Answer': a.response, 'Answer-Context Match': r_eval_result, 'Question-Answer-Context Match': qr_eval_result})
    time.sleep(8)

qa_match_df = pd.DataFrame(questions_and_answers)
qa_match_df


In [ ]:
import json

json_str = qa_match_df.to_json(orient='records', force_ascii=False, date_format='iso')

formatted_json_str = json.dumps(json.loads(json_str), indent=4)

with open('questions_answers_english_validation.json', 'w', encoding='utf-8') as wf:
    wf.write(formatted_json_str)

In [ ]:
qa_match_df[qa_match_df['Answer-Context Match'] == 'YES']

In [ ]:
qa_match_df[qa_match_df['Question-Answer-Context Match'] == 'YES']

### Evaluation Part 2: Questions in Vietnamese

##### Translate sampled questions to Vietnamese

In [ ]:
def gpt4_translate_to_vi(question):
    """
    Translate a question to Vietnamese using GPT-4 model.

    :param question: Input question
    :return: Translated question
    """
    messages = [
        {
            "role": "system",
            "content": f"Translate the following question delimited by triple backticks to Vietnamese: {question}"
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=1000,
        temperature=0)
    return response["choices"][0]["message"]["content"].strip()

In [ ]:
# Using loop instead of list comprehension because token per minute rate exceeded
sampled_vi_questions = []
for q in sampled_questions:
    sampled_vi_questions.append(gpt4_translate_to_vi(q))
    time.sleep(5)
sampled_vi_questions

In [ ]:
vi_questions_and_answers = []
for q in sampled_vi_questions:
    a = query_engine.query(q)
    r_eval_result = r_evaluator.evaluate(a)
    time.sleep(8)
    qr_eval_result = qr_evaluator.evaluate(q, a)
    vi_questions_and_answers.append({'Question': q, 'Answer': a.response, 'Answer-Context Match': r_eval_result, 'Question-Answer-Context Match': qr_eval_result})
    time.sleep(8)

vi_qa_match_df = pd.DataFrame(vi_questions_and_answers)
vi_qa_match_df

In [ ]:
json_str = vi_qa_match_df.to_json(orient='records', force_ascii=False, date_format='iso')

formatted_json_str = json.dumps(json.loads(json_str), indent=4)

with open('questions_answers_vietnamese_validation.json', 'w', encoding='utf-8') as wf:
    wf.write(formatted_json_str)

In [ ]:
vi_qa_match_df[vi_qa_match_df['Answer-Context Match'] == 'YES']

In [ ]:
vi_qa_match_df[vi_qa_match_df['Question-Answer-Context Match'] == 'YES']

In [ ]:
vi_qa_match_df[vi_qa_match_df['Answer-Context Match'] == 'NO']

### Evaluation Analysis

In [ ]:
qa_match_df.rename(columns={'Question': 'English Question', 'Answer': 'English Answer', 'Answer-Context Match': 'English Answer-Context Match', 'Question-Answer-Context Match': 'English Question-Answer-Context Match'}, inplace=True)
qa_match_df

In [ ]:
vi_qa_match_df.rename(columns={'Question': 'Vietnamese Question', 'Answer': 'Vietnamese Answer', 'Answer-Context Match': 'Vietnamese Answer-Context Match', 'Question-Answer-Context Match': 'Vietnamese Question-Answer-Context Match'}, inplace=True)
vi_qa_match_df

In [ ]:
agg_qa_match_df = pd.concat([qa_match_df, vi_qa_match_df], axis=1)
agg_qa_match_df

In [ ]:
largest_discrep_eng_vi_qa_df = agg_qa_match_df[(agg_qa_match_df['Vietnamese Answer-Context Match'] == 'NO') & (agg_qa_match_df['English Answer-Context Match'] == 'YES')]
largest_discrep_eng_vi_qa_df

In [ ]:
largest_discrep_eng_vi_ans_df = largest_discrep_eng_vi_qa_df[['English Answer', 'Vietnamese Answer']]
largest_discrep_eng_vi_ans_df

In [ ]:
def gpt4_translate_vi_to_en(text):
    """
    Translate from Vietnamese to English using GPT-4 model. Used for translating generated answers in Vietnamese.

    :param question: Input text
    :return: Translated text
    """
    messages = [
        {
            "role": "system",
            "content": f"Translate the following Vietnamese text delimited by triple backticks to English: {text}"
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=1000,
        temperature=0)
    return response["choices"][0]["message"]["content"].strip()

In [ ]:
translated_from_vi_to_en = []
for t in largest_discrep_eng_vi_ans_df['Vietnamese Answer'].tolist():
    translated_from_vi_to_en.append(gpt4_translate_vi_to_en(t))
    time.sleep(5)
largest_discrep_eng_vi_ans_df['English Translation of Vietnamese Answer'] = translated_from_vi_to_en
largest_discrep_eng_vi_ans_df

In [ ]:
def gpt4_check_texts_similar_meaning(text_one, text_two):
    """
    Checks if two bodies of text have the same meaning using GPT-4 model.

    :param question: Input text
    :return: Translated text
    """
    messages = [
        {
            "role": "system",
            "content": f"""On a scale of 1 to 10, where 1 means completely different and 10 means exactly the same,
            how similar in meaning are the following two bodies of text delimited by triple backticks?
            
            text_one: ```{text_one}```
            text_two: ```{text_two}"""
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=1000,
        temperature=0)
    return response["choices"][0]["message"]["content"].strip()

In [ ]:
similar_meaning = []
for en_ans, trans_vi_ans in zip(largest_discrep_eng_vi_ans_df['English Answer'].tolist(), largest_discrep_eng_vi_ans_df['English Translation of Vietnamese Answer']):
    sim_score = gpt4_check_texts_similar_meaning(en_ans, trans_vi_ans)
    if len(sim_score) > 0:
        similar_meaning.append(float(sim_score))
    else:
        similar_meaning.append(99999)
    time.sleep(5)
largest_discrep_eng_vi_ans_df['Similar Answer Score'] = similar_meaning
largest_discrep_eng_vi_ans_df

In [ ]:
from langdetect import detect

en_ans_source_lang, vi_ans_source_lang = [], []
for en_q, vi_q in zip(agg_qa_match_df['English Question'], agg_qa_match_df['Vietnamese Question']):
    en_ans = query_engine.query(en_q)
    src_langs = []
    for source_node in en_ans.source_nodes:
        src_langs.append(detect(source_node.node.text))
    en_ans_source_lang.append(src_langs)
    time.sleep(4)
    vi_ans = query_engine.query(vi_q)
    src_langs = []
    for source_node in vi_ans.source_nodes:
        src_langs.append(detect(source_node.node.text))
    vi_ans_source_lang.append(src_langs)
    time.sleep(4)
agg_qa_match_df['Source Language(s) of English Answer'] = en_ans_source_lang
agg_qa_match_df['Source Language(s) of Vietnamese Answer'] = vi_ans_source_lang
agg_qa_match_df

In [ ]:
for idx, row in agg_qa_match_df.iterrows():
    for l in row['Source Language(s) of English Answer']:
        if l != 'en':
            print('Context for English answer not in English for idx ', idx)
    for l in row['Source Language(s) of Vietnamese Answer']:
        if l != 'vi':
            print('Context for Vietnamese answer not in Vietnamese for idx ', idx)

In [ ]:
json_str = agg_qa_match_df.to_json(orient='records', force_ascii=False, date_format='iso')

formatted_json_str = json.dumps(json.loads(json_str), indent=4)

with open('questions_answers_aggregate_validation.json', 'w', encoding='utf-8') as wf:
    wf.write(formatted_json_str)

### Save Preliminary Result

In [ ]:
index.set_index_id("vector_index")
index.storage_context.persist('preliminary-llama-index')

### Save List of PDF Papers

In [ ]:
import os

filenames = []
dir_path = 'data/PDFs/'
for f in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, f)):
        filenames.append(f)

with open('extracted_files.txt', 'w') as wf:
    for f in filenames:
        wf.write(f + '\n')

### Sample English Question

In [ ]:
english_question = "What water supply issues exist in Dania Beach?"
query_engine = index.as_query_engine()
response = query_engine.query(english_question)
print(response)

### Sample Vietnamese Question

In [ ]:
vietnamese_question = """Ở đâu trong vùng Mê Công là nơi có mức độ xâm nhập mặn cao nhất trong mùa khô 2019-2020
và gây thiệt hại nghiêm trọng đối với các mô hình trồng cây ăn trái và nuôi trồng thủy sản?"""
query_engine = index.as_query_engine()
response = query_engine.query(vietnamese_question)
print(response)

### Load Index from Local Filesystem

In [ ]:
from llama_index import StorageContext, load_index_from_storage

sc = StorageContext.from_defaults(persist_dir='preliminary-llama-index')
index2 = load_index_from_storage(sc, 'vector_index')

In [ ]:
# Test same sample Vietnamese question from above
vietnamese_question = """Ở đâu trong vùng Mê Công là nơi có mức độ xâm nhập mặn cao nhất trong mùa khô 2019-2020
và gây thiệt hại nghiêm trọng đối với các mô hình trồng cây ăn trái và nuôi trồng thủy sản?"""
query_engine = index2.as_query_engine()
response = query_engine.query(vietnamese_question)
print(response)